In [38]:
# Inspect the first few rows
file_path = "/ca24/AstroCode-Collective/HD_209458_b_assignment3_taskA_spectrum.dat"
data = np.loadtxt(file_path)

print(data[:10])  # Print the first 10 rows


[[5.000168e+01 2.213571e-02 1.487807e-01]
 [4.999835e+01 2.229859e-02 1.493271e-01]
 [4.999502e+01 2.232564e-02 1.494177e-01]
 [4.999168e+01 2.230878e-02 1.493612e-01]
 [4.998835e+01 2.267096e-02 1.505688e-01]
 [4.998502e+01 2.278108e-02 1.509340e-01]
 [4.998169e+01 2.267005e-02 1.505658e-01]
 [4.997836e+01 2.247579e-02 1.499193e-01]
 [4.997502e+01 2.210697e-02 1.486841e-01]
 [4.997169e+01 2.210932e-02 1.486920e-01]]
